<a href="https://colab.research.google.com/github/siiddd/Risk-Analysis-Chatbot/blob/main/Final_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import re
import nltk

In [2]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [3]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [4]:
dataset = pd.read_csv('https://raw.githubusercontent.com/siiddd/Risk-Analysis-Chatbot/main/dataset_augmented.csv')
dataset = dataset.iloc[:, [-2, -1]]
dataset.head(3)

,Description,Accident Level
0,at level access scissor performed the sustaini...,0
1,in general store while two store attendants re...,0
2,being p m approximately in the nv in the tecno...,3


In [5]:
description = dataset.iloc[:, -2]
description.head(3)

0    at level access scissor performed the sustaini...
1    in general store while two store attendants re...
2    being p m approximately in the nv in the tecno...
Name: Description, dtype: object

In [6]:
words = []

for i in range(len(description)):
  dataset_word = nltk.word_tokenize(description[i])
  words = [*words, *dataset_word]
  del dataset_word

In [7]:
words = list(set(words))
len(words)

4657

In [9]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!rm glove.6B.zip glove.6B.300d.txt glove.6B.50d.txt glove.6B.100d.txt 

--2022-03-25 16:22:21--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-03-25 16:22:21--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-03-25 16:22:21--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [56]:
os.getcwd()

'/content'

In [10]:
embeddings = {}

f = open('glove.6B.200d.txt', 'r', encoding = 'utf-8')
for line in f:
  values = line.split()
  words = values[0]
  coefs = np.asarray(values[1:], dtype = 'float32')
  embeddings[words] = coefs
f.close()

In [11]:
data_list = list()
for i in range(len(description)):
    sentence = np.zeros(200)
    count = 0
    dataset_word = nltk.word_tokenize(description[i])
    for x in dataset_word:
        try:
            sentence += embeddings[x]
            count += 1
        except KeyError:
            continue
    data_list.append(sentence / count)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model_RF =RandomForestClassifier()

model_RF.fit(data_list, dataset['Accident Level'])

pred = model_RF.predict(data_list)

print('Random Forest \nTraining accuracy', accuracy_score(dataset['Accident Level'], pred))

Random Forest 
Training accuracy 0.9983844911147012


In [13]:
import xgboost as xgb

model_XGB = xgb.XGBClassifier()

model_XGB.fit(pd.DataFrame(data_list), dataset['Accident Level'])

pred = model_XGB.predict(pd.DataFrame(data_list))

print('XGB Classifier: \nTraining accuracy', accuracy_score(dataset['Accident Level'], pred))

XGB Classifier: 
Training accuracy 0.9959612277867528


In [14]:
from sklearn.svm import SVC

model_SVM = SVC(kernel = 'linear')

model_SVM.fit(data_list, dataset['Accident Level'])

pred = model_SVM.predict(data_list)

print('Support Vector Machine \nTraining accuracy', accuracy_score(dataset['Accident Level'], pred))

Support Vector Machine 
Training accuracy 0.851373182552504


In [28]:
import pickle
model = open("model_XGB.pkl","wb")
pickle.dump(model_XGB, model)
model.close()

In [15]:
def brazil(problem):
  embeddings = {}

  f = open('glove.6B.200d.txt', 'r', encoding = 'utf-8')

  for line in f:
    values = line.split()
    words = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    embeddings[words] = coefs
  f.close()

  data_list = list()
  sentence = np.zeros(200)
  count = 0
  dataset_word = nltk.word_tokenize(description[i])
  for x in dataset_word:
       try:
           sentence += embeddings[x]
           count += 1
       except KeyError:
           continue
  data_list.append(sentence / count)
  return data_list

In [34]:
x = brazil('i love mithra')
x

[array([ 1.81650094e-02,  5.28678655e-02, -1.54940734e-01, -1.78619997e-01,
         4.27634352e-02,  9.82692972e-02, -2.87257501e-01, -9.97086567e-02,
         5.59920593e-02,  3.92699345e-02,  1.04465355e-01,  1.91561497e-01,
         1.60464907e-01,  2.62077140e-01,  3.33544106e-01, -6.29350999e-02,
        -9.14926005e-02,  1.64357981e-01, -6.57374746e-02, -1.81931268e-01,
         2.84031399e-01,  2.51059766e+00,  3.99244316e-02, -1.23752867e-01,
         9.06416667e-02,  2.52985576e-02, -1.40242175e-01,  3.35377808e-02,
         3.67361985e-02, -2.87176016e-02, -5.61231330e-02, -7.60560819e-02,
         7.85480678e-02,  6.30617330e-02, -1.05598238e-03, -2.01342489e-01,
        -4.23438521e-01, -3.04349501e-01, -1.67737500e-01, -6.67068989e-02,
         2.30830074e-02, -1.73949364e-01, -3.14617315e-02,  2.61709714e-01,
        -1.28193667e-01,  1.09385734e-01,  1.24462477e-01,  7.11210753e-02,
        -2.24135728e-03,  2.13364564e-01, -4.83251654e-02,  7.76111061e-02,
        -7.5

In [29]:
pickle_in = open('model_XGB.pkl',"rb")

classifier = pickle.load(pickle_in)

TypeError: ignored